In [ ]:
#!sudo pip install requests
#!sudo pip install tabulate
#!sudo pip install scikit-learn
#!sudo pip install colorama
#!sudo pip install future

In [ ]:
# The following command removes the H2O module for Python.
#!sudo pip uninstall -y h2o

# Next, use pip to install this version of the H2O Python module.
#!sudo pip install http://h2o-release.s3.amazonaws.com/h2o/rel-weierstrass/2/Python/h2o-3.14.0.2-py2.py3-none-any.whl

In [ ]:
#!sudo pip install --upgrade requests

In [1]:
import h2o
import time
from h2o.estimators.deeplearning import H2OAutoEncoderEstimator, H2ODeepLearningEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator

In [ ]:
#!sudo apt-get update

In [ ]:
#!sudo apt-get install --assume-yes default-jre

In [ ]:
!java -version

In [2]:
# Start H2O on your local machine
h2o.init(nthreads=-1,min_mem_size = "50g")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_131"; OpenJDK Runtime Environment (build 1.8.0_131-8u131-b11-2ubuntu1.16.04.3-b11); OpenJDK 64-Bit Server VM (build 25.131-b11, mixed mode)
  Starting server from /usr/local/lib/python3.5/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpmbc_w_g2
  JVM stdout: /tmp/tmpmbc_w_g2/h2o_jeevananne_started_from_python.out
  JVM stderr: /tmp/tmpmbc_w_g2/h2o_jeevananne_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster version:,3.14.0.2
H2O cluster version age:,1 month
H2O cluster name:,H2O_from_python_jeevananne_opgcpg
H2O cluster total nodes:,1
H2O cluster free memory:,47.92 Gb
H2O cluster total cores:,22
H2O cluster allowed cores:,22
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


In [3]:
print("Import and Parse training data")
train = h2o.import_file(path='train.csv',col_types=["int"] + ['enum']+['string']+['enum']+['time']+['enum']*4+["string"]+['factor']+['string']+['numeric']+['enum']+['string']+['enum']+['factor'])

Import and Parse training data
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [63]:
print("Import and Parse testing data")
test = h2o.import_file(path='test.csv',col_types=['int']+['enum']+['string']+['enum']+['string']+['enum']*4+["string"]+['enum']+['string']+['enum']+['factor'])

Import and Parse testing data
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [33]:
train.names

['ID',
 'Country',
 'Carrier',
 'TrafficType',
 'ClickDate',
 'Device',
 'Browser',
 'OS',
 'UserIp',
 'ConversionPayOut',
 'publisherId',
 'advertiserCampaignId',
 'Fraud',
 'CountryMissing']

In [19]:
train = train.drop(['subPublisherId','ConversionDate','RefererUrl','ConversionStatus'])

In [68]:
test = test.drop(['subPublisherId','RefererUrl'])

In [39]:
#training feature engineering
train['CountryMissing'] = (train["Country"].isna()).ifelse(0, 1)
train['CountryMissing'] = (train["Country"]=='**').ifelse(0, 1)

train['TrafficTypeMissing'] = (train["TrafficType"].isna()).ifelse(0, 1)

train['DeviceMissing'] = (train["Device"].isna()).ifelse(0, 1)
train['BrowserMissing'] = (train["Browser"].isna()).ifelse(0, 1)
train['OSMissing'] = (train["OS"].isna()).ifelse(0, 1)

In [64]:
#testing feature engineering
test['CountryMissing'] = (test["Country"].isna()).ifelse(0, 1)
test['CountryMissing'] = (test["Country"]=='**').ifelse(0, 1)

test['TrafficTypeMissing'] = (test["TrafficType"].isna()).ifelse(0, 1)

test['DeviceMissing'] = (test["Device"].isna()).ifelse(0, 1)
test['BrowserMissing'] = (test["Browser"].isna()).ifelse(0, 1)
test['OSMissing'] = (test["OS"].isna()).ifelse(0, 1)

In [42]:
train[train['Country']=='**',:]

ID,Country,Carrier,TrafficType,ClickDate,Device,Browser,OS,UserIp,ConversionPayOut,publisherId,advertiserCampaignId,Fraud,CountryMissing,TrafficTypeMissing,DeviceMissing,BrowserMissing,OSMissing
568,**,-1,A,2017-08-21 00:00:07,Generic,android_webkit,Android,100.118.220.203,0,35,15752,0,0,1,1,1,1
6105,**,-1,M,2017-08-21 00:01:08,Generic,android_webkit,Android,100.97.28.136,0,437,3312,0,0,1,1,1,1
6874,**,-1,A,2017-08-21 00:01:16,Generic,android_webkit,Android,100.118.210.62,0,209,12230,0,0,1,1,1,1
7325,**,-1,A,2017-08-21 00:01:22,Generic,android_webkit,Android,100.96.25.136,0,1044,15751,0,0,1,1,1,1
8359,**,-1,A,2017-08-21 00:01:34,ZTE,chrome,Android,100.116.158.103,0,540,4128,0,0,1,1,1,1
9617,**,-1,M,2017-08-21 00:01:48,Generic,android_webkit,Android,100.97.165.163,0,1306,4788,0,0,1,1,1,1
10243,**,-1,M,2017-08-21 00:01:56,Generic,android_webkit,Android,100.96.196.245,0,1193,1643,0,0,1,1,1,1
11401,**,-1,A,2017-08-21 00:02:09,Lenovo,chrome,Android,100.118.18.134,0,1044,15751,0,0,1,1,1,1
12960,**,-1,,2017-08-21 00:02:28,Generic,android_webkit,Android,100.109.191.134,0,1306,7802,0,0,0,1,1,1
17113,**,-1,A,2017-08-21 00:03:16,Generic,android_webkit,Android,100.104.116.70,0,242,13959,0,0,1,1,1,1


In [45]:
def refine_date_col(data, col, pattern):
    #data[col]         = data[col].as_date(pattern) # As of 5/29/2106 H2O defaults parse as a date
    data["Day"]       = data[col].day()
    data["Month"]     = data[col].month() + 1     # Since H2O indexes from 0
    data["Year"]      = data[col].year() + 1900   # Start of epoch is 1900
    data["WeekNum"]   = data[col].week()
    data["WeekDay"]   = data[col].dayOfWeek()
    data["HourOfDay"] = data[col].hour()
    
    data.describe()  # HACK: Force evaluation before ifelse and cut. See PUBDEV-1425.
    
    # Create weekend and season cols
    # Spring = Mar, Apr, May. Summer = Jun, Jul, Aug. Autumn = Sep, Oct. Winter = Nov, Dec, Jan, Feb.
    # data["Weekend"]   = [1 if x in ("Sun", "Sat") else 0 for x in data["WeekDay"]]
    data["Weekend"] = ((data["WeekDay"] == "Sun") | (data["WeekDay"] == "Sat"))
    data["Season"] = data["Month"].cut([0, 2, 5, 7, 10, 12], ["Winter", "Spring", "Summer", "Autumn", "Winter"])
    
refine_date_col(train, "ClickDate", "%Y-%m-%d %H:%M:%S")
train = train.drop("ClickDate")
train.describe()

Rows:63367217
Cols:26




,ID,Country,Carrier,TrafficType,ClickDate,Device,Browser,OS,UserIp,ConversionPayOut,publisherId,advertiserCampaignId,Fraud,CountryMissing,TrafficTypeMissing,DeviceMissing,BrowserMissing,OSMissing,Day,Month,Year,WeekNum,WeekDay,HourOfDay,Weekend,Season
type,int,enum,string,enum,time,enum,enum,enum,string,real,enum,enum,enum,int,int,int,int,int,int,int,int,int,enum,int,int,enum
mins,0.0,,NaN,,1503273601000.0,,,,NaN,0.0,,,,0.0,0.0,0.0,0.0,0.0,21.0,9.0,3917.0,34.0,,0.0,0.0,
mean,31683608.0,,NaN,,1503692165288.9578,,,,NaN,0.032880682200727825,,,,0.9996982035679427,0.8502150725666239,0.9979028430426414,0.9956939406065443,0.995125192258325,25.296599280981518,9.0,3917.0,34.27810197187609,,12.650071424156133,0.18352598631560543,
maxs,63367216.0,,NaN,,1504137599000.0,,,,NaN,8000.0,,,,1.0,1.0,1.0,1.0,1.0,30.0,9.0,3917.0,35.0,,23.0,1.0,
sigma,18292540.04071128,,NaN,,249274564.98401314,,,,NaN,7.0352244079349395,,,,0.017369667692046044,0.3568604838825101,0.04574668209914968,0.06547913647580611,0.06964943695186046,2.880933315124209,0.0,0.0,0.448063910935639,,6.925952734967803,0.3870971467567096,
zeros,1,,0,,0,,,,0,63332693,,,,19124,9491454,132891,272863,308903,0,0,0,0,,2740559,51737686,
missing,0,41059,0,9491454,0,132891,272863,308903,0,2,199,460,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0.0,IN,2.0,M,2017-08-21 00:00:01,Google,chrome,Android,42.111.171.131,0.0,69,6555.0,0.0,1.0,1.0,1.0,1.0,1.0,21.0,9.0,3917.0,34.0,Mon,0.0,0.0,Autumn
1,1.0,TH,27.0,A,2017-08-21 00:00:01,Generic,46.0.2490.76,Android,1.46.235.65,0.0,646,7510.0,0.0,1.0,1.0,1.0,1.0,1.0,21.0,9.0,3917.0,34.0,Mon,0.0,0.0,Autumn
2,2.0,IN,4.0,M,2017-08-21 00:00:01,Samsung,chromium,Android,49.15.91.224,0.0,301,12696.0,0.0,1.0,1.0,1.0,1.0,1.0,21.0,9.0,3917.0,34.0,Mon,0.0,0.0,Autumn


Rows:63367217
Cols:25




,ID,Country,Carrier,TrafficType,Device,Browser,OS,UserIp,ConversionPayOut,publisherId,advertiserCampaignId,Fraud,CountryMissing,TrafficTypeMissing,DeviceMissing,BrowserMissing,OSMissing,Day,Month,Year,WeekNum,WeekDay,HourOfDay,Weekend,Season
type,int,enum,string,enum,enum,enum,enum,string,real,enum,enum,enum,int,int,int,int,int,int,int,int,int,enum,int,int,enum
mins,0.0,,NaN,,,,,NaN,0.0,,,,0.0,0.0,0.0,0.0,0.0,21.0,9.0,3917.0,34.0,,0.0,0.0,
mean,31683608.0,,NaN,,,,,NaN,0.032880682200727825,,,,0.9996982035679427,0.8502150725666239,0.9979028430426414,0.9956939406065443,0.995125192258325,25.296599280981518,9.0,3917.0,34.27810197187609,,12.650071424156133,0.18352598631560543,
maxs,63367216.0,,NaN,,,,,NaN,8000.0,,,,1.0,1.0,1.0,1.0,1.0,30.0,9.0,3917.0,35.0,,23.0,1.0,
sigma,18292540.04071128,,NaN,,,,,NaN,7.0352244079349395,,,,0.017369667692046044,0.3568604838825101,0.04574668209914968,0.06547913647580611,0.06964943695186046,2.880933315124209,0.0,0.0,0.448063910935639,,6.925952734967803,0.3870971467567096,
zeros,1,,0,,,,,0,63332693,,,,19124,9491454,132891,272863,308903,0,0,0,0,,2740559,51737686,
missing,0,41059,0,9491454,132891,272863,308903,0,2,199,460,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,0.0,IN,2.0,M,Google,chrome,Android,42.111.171.131,0.0,69,6555.0,0.0,1.0,1.0,1.0,1.0,1.0,21.0,9.0,3917.0,34.0,Mon,0.0,0.0,Autumn
1,1.0,TH,27.0,A,Generic,46.0.2490.76,Android,1.46.235.65,0.0,646,7510.0,0.0,1.0,1.0,1.0,1.0,1.0,21.0,9.0,3917.0,34.0,Mon,0.0,0.0,Autumn
2,2.0,IN,4.0,M,Samsung,chromium,Android,49.15.91.224,0.0,301,12696.0,0.0,1.0,1.0,1.0,1.0,1.0,21.0,9.0,3917.0,34.0,Mon,0.0,0.0,Autumn


In [65]:
def refine_date_col(data, col, pattern):
    data[col]         = data[col].as_date(pattern) # As of 5/29/2106 H2O defaults parse as a date
    data["Day"]       = data[col].day()
    data["Month"]     = data[col].month() + 1     # Since H2O indexes from 0
    data["Year"]      = data[col].year() + 1900   # Start of epoch is 1900
    data["WeekNum"]   = data[col].week()
    data["WeekDay"]   = data[col].dayOfWeek()
    data["HourOfDay"] = data[col].hour()
    
    data.describe()  # HACK: Force evaluation before ifelse and cut. See PUBDEV-1425.
    
    # Create weekend and season cols
    # Spring = Mar, Apr, May. Summer = Jun, Jul, Aug. Autumn = Sep, Oct. Winter = Nov, Dec, Jan, Feb.
    # data["Weekend"]   = [1 if x in ("Sun", "Sat") else 0 for x in data["WeekDay"]]
    data["Weekend"] = ((data["WeekDay"] == "Sun") | (data["WeekDay"] == "Sat"))
    data["Season"] = data["Month"].cut([0, 2, 5, 7, 10, 12], ["Winter", "Spring", "Summer", "Autumn", "Winter"])
    
refine_date_col(test, "ClickDate", "%Y-%m-%d %H:%M:%S")
test = test.drop("ClickDate")
test.describe()

Rows:25548873
Cols:25




,ID,Country,Carrier,TrafficType,ClickDate,Device,Browser,OS,RefererUrl,UserIp,publisherId,subPublisherId,advertiserCampaignId,Fraud,CountryMissing,TrafficTypeMissing,DeviceMissing,BrowserMissing,OSMissing,Day,Month,Year,WeekNum,WeekDay,HourOfDay
type,int,enum,string,enum,int,enum,enum,enum,enum,string,enum,string,enum,enum,int,int,int,int,int,int,int,int,int,enum,int
mins,63367289.0,,NaN,,1504137601000.0,,,,,NaN,,NaN,,,0.0,0.0,0.0,0.0,0.0,1.0,9.0,3917.0,35.0,,0.0
mean,76141725.0,,NaN,,1504315279248.3716,,,,,NaN,,NaN,,,0.9997809296715358,0.796449769036779,0.9978568917697466,0.9962779962936135,0.9947876370124037,8.862143703951254,9.763068414015756,3917.0,35.0,,12.440915652130725
maxs,88916161.0,,NaN,,1504483139000.0,,,,,NaN,,NaN,,,1.0,1.0,1.0,1.0,1.0,31.0,10.0,3917.0,35.0,,23.0
sigma,7375324.496358347,,NaN,,97551760.01795556,,,,,NaN,,NaN,,,0.014799403542983168,0.4026382256858252,0.04624408503873537,0.06089458547305925,0.07200829440164433,12.356133711190422,0.4251999725113753,0.0,0.0,,6.850438040764894
zeros,0,,0,,0,,,,,0,,0,,,5597,5200479,54754,95093,133170,0,0,0,0,,1091449
missing,0,9038,0,5200479,0,54754,95093,133170,16309125,0,152,24856158,398,0,0,0,0,0,0,0,0,0,0,0,0
0,63367289.0,IN,5.0,,1504137601000.0,Generic,uc,Android,,101.58.211.15,1479,,16216.0,0.0,1.0,0.0,1.0,1.0,1.0,31.0,9.0,3917.0,35.0,Thu,0.0
1,63367290.0,IN,2.0,M,1504137601000.0,Datawind,chrome,Android,,42.107.175.145,3734,,1836.0,0.0,1.0,1.0,1.0,1.0,1.0,31.0,9.0,3917.0,35.0,Thu,0.0
2,63367291.0,IN,1.0,A,1504137601000.0,Panasonic,chrome,Android,synctraff.com,106.223.85.251,282.0,,13431.0,0.0,1.0,1.0,1.0,1.0,1.0,31.0,9.0,3917.0,35.0,Thu,0.0


Rows:25548873
Cols:26




,ID,Country,Carrier,TrafficType,Device,Browser,OS,RefererUrl,UserIp,publisherId,subPublisherId,advertiserCampaignId,Fraud,CountryMissing,TrafficTypeMissing,DeviceMissing,BrowserMissing,OSMissing,Day,Month,Year,WeekNum,WeekDay,HourOfDay,Weekend,Season
type,int,enum,string,enum,enum,enum,enum,enum,string,enum,string,enum,enum,int,int,int,int,int,int,int,int,int,enum,int,int,enum
mins,63367289.0,,NaN,,,,,,NaN,,NaN,,,0.0,0.0,0.0,0.0,0.0,1.0,9.0,3917.0,35.0,,0.0,0.0,
mean,76141725.0,,NaN,,,,,,NaN,,NaN,,,0.9997809296715358,0.796449769036779,0.9978568917697466,0.9962779962936135,0.9947876370124037,8.862143703951254,9.763068414015756,3917.0,35.0,,12.440915652130725,0.5068389122291226,
maxs,88916161.0,,NaN,,,,,,NaN,,NaN,,,1.0,1.0,1.0,1.0,1.0,31.0,10.0,3917.0,35.0,,23.0,1.0,
sigma,7375324.496358347,,NaN,,,,,,NaN,,NaN,,,0.014799403542983168,0.4026382256858252,0.04624408503873537,0.06089458547305925,0.07200829440164433,12.356133711190422,0.4251999725113753,0.0,0.0,,6.850438040764894,0.4999532368760698,
zeros,0,,0,,,,,,0,,0,,,5597,5200479,54754,95093,133170,0,0,0,0,,1091449,12599710,
missing,0,9038,0,5200479,54754,95093,133170,16309125,0,152,24856158,398,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,63367289.0,IN,5.0,,Generic,uc,Android,,101.58.211.15,1479,,16216.0,0.0,1.0,0.0,1.0,1.0,1.0,31.0,9.0,3917.0,35.0,Thu,0.0,0.0,Autumn
1,63367290.0,IN,2.0,M,Datawind,chrome,Android,,42.107.175.145,3734,,1836.0,0.0,1.0,1.0,1.0,1.0,1.0,31.0,9.0,3917.0,35.0,Thu,0.0,0.0,Autumn
2,63367291.0,IN,1.0,A,Panasonic,chrome,Android,synctraff.com,106.223.85.251,282.0,,13431.0,0.0,1.0,1.0,1.0,1.0,1.0,31.0,9.0,3917.0,35.0,Thu,0.0,0.0,Autumn


In [66]:
test

ID,Country,Carrier,TrafficType,Device,Browser,OS,RefererUrl,UserIp,publisherId,subPublisherId,advertiserCampaignId,Fraud,CountryMissing,TrafficTypeMissing,DeviceMissing,BrowserMissing,OSMissing,Day,Month,Year,WeekNum,WeekDay,HourOfDay,Weekend,Season
6.33673e+07,IN,5,,Generic,uc,Android,,101.58.211.15,1479,,16216,0,1,0,1,1,1,31,9,3917,35,Thu,0,0,Autumn
6.33673e+07,IN,2,M,Datawind,chrome,Android,,42.107.175.145,3734,,1836,0,1,1,1,1,1,31,9,3917,35,Thu,0,0,Autumn
6.33673e+07,IN,1,A,Panasonic,chrome,Android,synctraff.com,106.223.85.251,282,,13431,0,1,1,1,1,1,31,9,3917,35,Thu,0,0,Autumn
6.33673e+07,BR,-1,M,Generic,android_webkit,Android,,177.35.22.162,1203,,4787,0,1,1,1,1,1,31,9,3917,35,Thu,0,0,Autumn
6.33673e+07,IN,3,A,Micromax,chrome,Android,,101.210.255.232,597,,9769,0,1,1,1,1,1,31,9,3917,35,Thu,0,0,Autumn
6.33673e+07,BD,45,,Symphony,46.0.2490.76,Android,,103.67.158.77,1306,,10227,0,1,0,1,1,1,31,9,3917,35,Thu,0,0,Autumn
6.33673e+07,RU,-1,A,Micromax,46.0.2490.76,Android,,46.146.8.55,55,,11225,0,1,1,1,1,1,31,9,3917,35,Thu,0,0,Autumn
6.33673e+07,ID,47,M,Generic,android_webkit,Android,,120.188.84.62,55,,13960,0,1,1,1,1,1,31,9,3917,35,Thu,0,0,Autumn
6.33673e+07,ID,46,M,Generic,android_webkit,Android,,112.215.244.33,1306,,2024,0,1,1,1,1,1,31,9,3917,35,Thu,0,0,Autumn
6.33673e+07,US,916,,Generic,android_webkit,Android,,172.58.70.188,1306,,12345,0,1,0,1,1,1,31,9,3917,35,Thu,0,0,Autumn


In [69]:
test.describe()

Rows:25548873
Cols:24




,ID,Country,Carrier,TrafficType,Device,Browser,OS,UserIp,publisherId,advertiserCampaignId,Fraud,CountryMissing,TrafficTypeMissing,DeviceMissing,BrowserMissing,OSMissing,Day,Month,Year,WeekNum,WeekDay,HourOfDay,Weekend,Season
type,int,enum,string,enum,enum,enum,enum,string,enum,enum,enum,int,int,int,int,int,int,int,int,int,enum,int,int,enum
mins,63367289.0,,NaN,,,,,NaN,,,,0.0,0.0,0.0,0.0,0.0,1.0,9.0,3917.0,35.0,,0.0,0.0,
mean,76141725.0,,NaN,,,,,NaN,,,,0.9997809296715358,0.796449769036779,0.9978568917697466,0.9962779962936135,0.9947876370124037,8.862143703951254,9.763068414015756,3917.0,35.0,,12.440915652130725,0.5068389122291226,
maxs,88916161.0,,NaN,,,,,NaN,,,,1.0,1.0,1.0,1.0,1.0,31.0,10.0,3917.0,35.0,,23.0,1.0,
sigma,7375324.496358347,,NaN,,,,,NaN,,,,0.014799403542983168,0.4026382256858252,0.04624408503873537,0.06089458547305925,0.07200829440164433,12.356133711190422,0.4251999725113753,0.0,0.0,,6.850438040764894,0.4999532368760698,
zeros,0,,0,,,,,0,,,,5597,5200479,54754,95093,133170,0,0,0,0,,1091449,12599710,
missing,0,9038,0,5200479,54754,95093,133170,0,152,398,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,63367289.0,IN,5.0,,Generic,uc,Android,101.58.211.15,1479,16216.0,0.0,1.0,0.0,1.0,1.0,1.0,31.0,9.0,3917.0,35.0,Thu,0.0,0.0,Autumn
1,63367290.0,IN,2.0,M,Datawind,chrome,Android,42.107.175.145,3734,1836.0,0.0,1.0,1.0,1.0,1.0,1.0,31.0,9.0,3917.0,35.0,Thu,0.0,0.0,Autumn
2,63367291.0,IN,1.0,A,Panasonic,chrome,Android,106.223.85.251,282.0,13431.0,0.0,1.0,1.0,1.0,1.0,1.0,31.0,9.0,3917.0,35.0,Thu,0.0,0.0,Autumn


In [76]:
# ----------
# 3- Fit a model on train; using test as validation

# Function for doing class test/train/holdout split
def split_fit_predict(data):
  global gbm0,drf0,glm0,dl0
  # Classic Test/Train split
  r = data['ID'].runif()   # Random UNIForm numbers, one per row
  train = data[  r  < 0.6]
  test  = data[(0.6 <= r) & (r < 0.9)]
  hold  = data[ 0.9 <= r ]
  print("Training data has",train.ncol,"columns and",train.nrow,"rows, test has",test.nrow,"rows, holdout has",hold.nrow)
  avclickprednames_x = data.names
  avclickprednames_x.remove("ConversionPayOut")
  avclickprednames_x.remove("ID")
  # Run GBM
  s = time.time()
  
  gbm0 = H2OGradientBoostingEstimator(ntrees=500, # 500 works well
                                      max_depth=6,
                                      learn_rate=0.1)
    

  gbm0.train(x               =avclickprednames_x,
             y               ="ConversionPayOut",
             training_frame  =train,
             validation_frame=test)

  gbm_elapsed = time.time() - s

  # Run DRF
  s = time.time()
    
  drf0 = H2ORandomForestEstimator(ntrees=250, max_depth=30)

  drf0.train(x               =avclickprednames_x,
             y               ="ConversionPayOut",
             training_frame  =train,
             validation_frame=test)
    
  drf_elapsed = time.time() - s 
    
    
  # Run GLM
  #if "WC1" in bike_names_x: bike_names_x.remove("WC1")
  s = time.time()

  glm0 = H2OGeneralizedLinearEstimator(Lambda=[1e-5], family="poisson")
    
  glm0.train(x               =avclickprednames_x,
             y               ="ConversionPayOut",
             training_frame  =train,
             validation_frame=test)

  glm_elapsed = time.time() - s
  
  # Run DL
  s = time.time()

  dl0 = H2ODeepLearningEstimator(hidden=[10,10,10,10], epochs=50)
    
  dl0.train(x               =avclickprednames_x,
            y               ="ConversionPayOut",
            training_frame  =train,
            validation_frame=test)
    
  dl_elapsed = time.time() - s
  
  # ----------
  # 4- Score on holdout set & report
  train_mse_gbm = gbm0.model_performance(train).mse()
  test_mse_gbm  = gbm0.model_performance(test ).mse()
  hold_mse_gbm  = gbm0.model_performance(hold ).mse()
#   print "GBM mse TRAIN=",train_mse_gbm,", mse TEST=",test_mse_gbm,", mse HOLDOUT=",hold_mse_gbm
  
  train_mse_drf = drf0.model_performance(train).mse()
  test_mse_drf  = drf0.model_performance(test ).mse()
  hold_mse_drf  = drf0.model_performance(hold ).mse()
#   print "DRF mse TRAIN=",train_mse_drf,", mse TEST=",test_mse_drf,", mse HOLDOUT=",hold_mse_drf
  
  train_mse_glm = glm0.model_performance(train).mse()
  test_mse_glm  = glm0.model_performance(test ).mse()
  hold_mse_glm  = glm0.model_performance(hold ).mse()
#   print "GLM mse TRAIN=",train_mse_glm,", mse TEST=",test_mse_glm,", mse HOLDOUT=",hold_mse_glm
    
  train_mse_dl = dl0.model_performance(train).mse()
  test_mse_dl  = dl0.model_performance(test ).mse()
  hold_mse_dl  = dl0.model_performance(hold ).mse()
#   print " DL mse TRAIN=",train_mse_dl,", mse TEST=",test_mse_dl,", mse HOLDOUT=",hold_mse_dl
    
  # make a pretty HTML table printout of the results

  header = ["Model", "mse TRAIN", "mse TEST", "mse HOLDOUT", "Model Training Time (s)"]
  table  = [
            ["GBM", train_mse_gbm, test_mse_gbm, hold_mse_gbm, round(gbm_elapsed,3)],
            ["DRF", train_mse_drf, test_mse_drf, hold_mse_drf, round(drf_elapsed,3)],
            ["GLM", train_mse_glm, test_mse_glm, hold_mse_glm, round(glm_elapsed,3)],
            ["DL ", train_mse_dl,  test_mse_dl,  hold_mse_dl , round(dl_elapsed,3) ],
           ]
  h2o.display.H2ODisplay(table,header)
  # --------------

In [72]:
dd = train

In [ ]:
glm0 = H2OGeneralizedLinearEstimator(Lambda=[1e-5], family="poisson")
    
  glm0.train(x               =avclickprednames_x,
             y               ="ConversionPayOut",
             training_frame  =train,
             validation_frame=test)


In [80]:
glm0

Model Details
H2OGeneralizedLinearEstimator :  Generalized Linear Modeling
Model Key:  GLM_model_python_1506011317762_3


ModelMetricsRegressionGLM: glm
** Reported on train data. **

MSE: 46.12669566883413
RMSE: 6.79166368932047
MAE: 0.06390996033754881
RMSLE: 0.11508928899381601
R^2: 0.005304504253967113
Mean Residual Deviance: 0.46059105776732145
Null degrees of freedom: 38019910
Residual degrees of freedom: 38019409
Null deviance: 23376607.300322488
Residual deviance: 17511631.02370942
AIC: 17598975.173277967

ModelMetricsRegressionGLM: glm
** Reported on validation data. **

MSE: 48.60320893450031
RMSE: 6.971600170298086
MAE: 0.06575745314646207
RMSLE: 0.11573790988762478
R^2: -0.00024192934461475524
Mean Residual Deviance: 0.5033904393032442
Null degrees of freedom: 19010511
Residual degrees of freedom: 19010010
Null deviance: 12301236.135290146
Residual deviance: 9569709.987059595
AIC: 9614431.037239041
Scoring History: 


,timestamp,duration,iterations,negative_log_likelihood,objective
,2017-09-21 19:12:20,0.000 sec,0,23376607.2958022,0.6148517
,2017-09-21 19:12:31,11.430 sec,1,21662151.1719286,0.5701414
,2017-09-21 19:12:37,17.030 sec,2,19605635.9918504,0.5188371
,2017-09-21 19:12:44,24.302 sec,3,17996200.4362450,0.4773787
,2017-09-21 19:12:49,28.528 sec,4,17599762.1845317,0.4674633
,2017-09-21 19:12:53,32.770 sec,5,17519142.8517779,0.4655724
,2017-09-21 19:12:57,36.933 sec,6,17514393.3228728,0.4654644
,2017-09-21 19:13:01,41.121 sec,7,17511631.0191996,0.4654100


In [81]:
gbm0

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1506011317762_1


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 46.31748368263441
RMSE: 6.8056949448703925
MAE: 0.06400482691049321
RMSLE: NaN
Mean Residual Deviance: 46.31748368263441

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 48.59064526621538
RMSE: 6.970699051473631
MAE: 0.06570888390341172
RMSLE: NaN
Mean Residual Deviance: 48.59064526621538
Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2017-09-21 18:43:45,0.015 sec,0.0,6.8097489,0.0640326,46.3726797,6.9707572,0.0656942,48.5914560
,2017-09-21 18:44:11,25.290 sec,3.0,6.8076913,0.0640168,46.3446611,6.9704456,0.0656951,48.5871120
,2017-09-21 18:44:50,1 min 4.590 sec,28.0,6.8056949,0.0640048,46.3174837,6.9706991,0.0657089,48.5906453
,2017-09-21 18:46:08,2 min 22.659 sec,86.0,6.8056949,0.0640048,46.3174837,6.9706991,0.0657089,48.5906453
,2017-09-21 18:49:13,5 min 27.390 sec,223.0,6.8056949,0.0640048,46.3174837,6.9706991,0.0657089,48.5906453
,2017-09-21 18:55:37,11 min 51.343 sec,500.0,6.8056949,0.0640048,46.3174837,6.9706991,0.0657089,48.5906453


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
WeekDay,4055014.0,1.0,0.3671358
publisherId,1778146.7500000,0.4385057,0.1609912
Device,1414131.2500000,0.3487365,0.1280336
HourOfDay,1337111.5,0.3297428,0.1210604
advertiserCampaignId,920232.0625000,0.2269368,0.0833166
Browser,406388.7812500,0.1002188,0.0367939
Day,391504.0,0.0965481,0.0354463
TrafficTypeMissing,263466.25,0.0649730,0.0238539
Country,246168.2656250,0.0607071,0.0222878
TrafficType,232603.5312500,0.0573620,0.0210596


In [82]:
drf0

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  DRF_model_python_1506011317762_2


ModelMetricsRegression: drf
** Reported on train data. **

MSE: 46.411636607418465
RMSE: 6.812608649219363
MAE: 0.0640074640068397
RMSLE: 0.07558442622454105
Mean Residual Deviance: 46.411636607418465

ModelMetricsRegression: drf
** Reported on validation data. **

MSE: 48.64824670872802
RMSE: 6.974829511086849
MAE: 0.06575510863758963
RMSLE: 0.07627097969901725
Mean Residual Deviance: 48.64824670872802
Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2017-09-21 18:57:20,0.007 sec,0.0,nan,nan,nan,nan,nan,nan
,2017-09-21 18:57:23,2.894 sec,1.0,6.4417222,0.0644708,41.4957848,6.9707572,0.0656942,48.5914560
,2017-09-21 18:57:39,18.948 sec,4.0,6.8258185,0.0641423,46.5917980,6.9876073,0.0656613,48.8266556
,2017-09-21 18:58:05,44.576 sec,11.0,6.8283979,0.0641023,46.6270180,6.9811404,0.0658081,48.7363217
,2017-09-21 18:58:50,1 min 30.019 sec,27.0,6.8247766,0.0641278,46.5775755,6.9790859,0.0658414,48.7076395
,2017-09-21 19:00:16,2 min 55.973 sec,47.0,6.8213037,0.0640833,46.5301842,6.9809690,0.0658491,48.7339275
,2017-09-21 19:02:33,5 min 13.046 sec,89.0,6.8158357,0.0640137,46.4556162,6.9775746,0.0658010,48.6865475
,2017-09-21 19:06:31,9 min 11.405 sec,167.0,6.8129519,0.0640116,46.4163133,6.9749202,0.0657567,48.6495115
,2017-09-21 19:11:15,13 min 54.475 sec,250.0,6.8126086,0.0640075,46.4116366,6.9748295,0.0657551,48.6482467


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
HourOfDay,1345172224.0000000,1.0,0.3148284
Device,923930240.0000000,0.6868490,0.2162396
publisherId,642722048.0000000,0.4777991,0.1504247
advertiserCampaignId,383108864.0000000,0.2848028,0.0896640
WeekDay,324755104.0000000,0.2414227,0.0760067
Browser,221572992.0000000,0.1647172,0.0518576
Day,169023712.0000000,0.1256521,0.0395588
Country,120381432.0000000,0.0894915,0.0281745
TrafficType,51323152.0000000,0.0381536,0.0120118
WeekNum,49267552.0000000,0.0366255,0.0115307


In [83]:
preds = gbm0.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


/usr/local/lib/python3.5/dist-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'Device' has levels not trained on: [AEG, ASK, Altron, BigBen, DQV, Desay, Digital2, Easyteck, Ferguson, GUOMI, Gadnic, Haipai, Hott, Kingplay, MWG, Microvission, Mimo, Mobo, MouseComputer, MyMobi, Nevitel, Novarra, Oaks, Ola, Omega, Opal, PENTAGRAM, Pierre Cardin, Red-X, Rikomagic, Rio Technology, S Nexian, SUMVISION, Semp, Simmtronics, Skaynet, TBS, Tracer, Victor, Windows Mobile, myPad]
  warnings.warn(w)
/usr/local/lib/python3.5/dist-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'Browser' has levels not trained on: [palm]
  warnings.warn(w)
/usr/local/lib/python3.5/dist-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'OS' has levels not trained on: [Palm OS]
  warnings.warn(w)
/usr/local/lib/python3.5/dist-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'publisherId' has levels not trained on: [1024, 10530.0, 10536, 10690, 1

/usr/local/lib/python3.5/dist-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'advertiserCampaignId' has levels not trained on: [10586.0, 10671.0, 10937.0, 11012.0, 11041.0, 11111.0, 11182.0, 11321.0, 11907.0, 12167.0, 12350.0, 12399.0, 12406.0, 12533.0, 12983.0, 13012.0, 13291.0, 13595.0, 13772.0, 13773.0, 13775.0, 13790.0, 13969.0, 13970.0, 13989.0, 14001.0, 14217.0, 14251.0, 14373.0, 14404.0, 14426.0, 14554.0, 14683.0, 14788.0, 14874.0, 14875.0, 14898.0, 14930.0, 15038.0, 15046.0, 15209.0, 15280.0, 15379.0, 15482.0, 15500.0, 15502.0, 15516.0, 15521.0, 15659.0, 15712.0, 15721.0, 15723.0, 15801.0, 15867.0, 15893.0, 15951.0, 16094.0, 16183.0, 16225.0, 16263.0, 16346.0, 16375.0, 16418.0, 16503.0, 16555.0, 16577.0, 16585.0, 16596.0, 16626.0, 16652.0, 16692.0, 16714.0, 16716.0, 16717.0, 16718.0, 16752.0, 16949.0, 17029.0, 17109.0, 17153.0, 17207.0, 17391.0, 1747.0, 17519.0, 17541.0, 17550.0, 17623.0, 17647.0, 17648.0, 17654.0, 17708.0, 17709.0, 17710.0, 17711.0, 17712.

In [84]:
preds

predict
0.0398266
0.0398266
0.0398266
0.0241787
0.0398266
0.0283167
0.0438346
0.0246382
0.0246382
0.0238966


In [86]:
preds['predict']

predict
0.0398266
0.0398266
0.0398266
0.0241787
0.0398266
0.0283167
0.0438346
0.0246382
0.0246382
0.0238966


In [88]:
import pandas as pd

In [90]:
submission = pd.concat((h2o.as_list(test['ID']), h2o.as_list(preds)), axis=1, ignore_index=True)
submission.columns = ['ID', 'ConversionPayOut']
submission = submission.set_index(['ID', 'ConversionPayOut'])
submission.to_csv('submission_h2o_gbm.csv')

In [92]:
!gsutil -m cp submission_h2o_gbm.csv gs://avclickprediction/

Copying file://submission_h2o_gbm.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

\ [1/1 files][705.6 MiB/705.6 MiB] 100% Done                                    
Operation completed over 1 objects/705.6 MiB.                                    


In [94]:
!head -n 5 submission_h2o_gbm.csv

ID,ConversionPayOut
63367289,0.0398266008331099
63367290,0.0398266008331099
63367291,0.0398266008331099
63367292,0.02417865560600845


In [95]:
!head -n 5 test.csv

ID,Country,Carrier,TrafficType,ClickDate,Device,Browser,OS,RefererUrl,UserIp,publisherId,subPublisherId,advertiserCampaignId,Fraud
63367289,IN,5.0,,2017-08-31 00:00:01,Generic,uc,Android,,101.58.211.15,1479,,16216.0,0.0
63367290,IN,2.0,M,2017-08-31 00:00:01,Datawind,chrome,Android,,42.107.175.145,3734,,1836.0,0.0
63367291,IN,1.0,A,2017-08-31 00:00:01,Panasonic,chrome,Android,synctraff.com,106.223.85.251,282.0,,13431.0,0.0
63367292,BR,-1.0,M,2017-08-31 00:00:01,Generic,android_webkit,Android,,177.35.22.162,1203,,4787.0,0.0


In [4]:
!tail -n 5 submission_h2o_gbm.csv

88916157,0.026864922276943308
88916158,0.024434301091603317
88916159,0.0398266008331099
88916160,0.0398266008331099
88916161,0.024385130519292638


In [5]:
!tail -n 5 train.csv

63367212,BD,-1.0,,2017-08-30 23:59:59,Generic,android_webkit,Android,,103.73.224.162,False,,0.0,1306,,6113.0,0.0
63367213,IN,121.0,A,2017-08-30 23:59:59,Lenovo,chromium,Android,overtraff.com,49.34.99.190,False,,0.0,282.0,,11991.0,0.0
63367214,VE,-1.0,A,2017-08-30 23:59:59,Niu,chrome,Android,track.cpa.tapgerine.com,186.185.236.137,False,,0.0,1362.0,,13960.0,0.0
63367215,IN,1.0,M,2017-08-30 23:59:59,Lava,46.0.2490.76,Android,,106.208.117.28,False,,0.0,209,,5697.0,0.0
63367216,RU,-1.0,M,2017-08-30 23:59:59,Apple,safari,iOS,koolmedia.info,176.124.144.185,False,,0.0,1206,,11830.0,0.0
